In [ ]:
!pip install braindecode moabb ncps

In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import create_windows_from_events

In [2]:
dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids = [i for i in range(1, 10)])

from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
    ),
]

# Preprocess the data
preprocess(dataset, preprocessors, n_jobs=-1)
trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

/usr/local/lib/python3.11/dist-packages/moabb/datasets/download.py:56: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BNCI_PATH"
  set_config(key, get_config("MNE_DATA"))


MNE_DATA is not already configured. It will be set to default location in the home directory - /root/mne_data
All datasets will be downloaded to this location, if anything is already downloaded, please move manually to this location


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lampx.tugraz.at'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|█████████████████████████████████████| 42.8M/42.8M [00:00<00:00, 39.7GB/s]
SHA256 hash of downloaded file: 054f02e70cf9c4ada1517e9b9864f45407939c1062c6793516585c6f511d0325
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lampx.tugraz.at'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|█████████████████████████████████████| 43.8M/43.8M [

In [ ]:
# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)
splitted = windows_dataset.split("session")
train_set = splitted['0train']  # Session train
test_set = splitted['1test']  # Session evaluation
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size)
progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
for batch_idx, (X, y, _) in progress_bar:
  print(X.shape, y.shape)
  print(y)
  break


In [4]:
class EEGWindowDataset(torch.utils.data.Dataset):
    def __init__(self, bd_dataset):
        self.bd_dataset = bd_dataset

    def __len__(self):
        return len(self.bd_dataset)

    def __getitem__(self, idx):
        X, y, _ = self.bd_dataset[idx]
        # X: (channels, time) -> (time, channels)
        X = torch.from_numpy(X).float().permute(1, 0)
        y = torch.tensor(y).long()
        return X, y

In [5]:
train_dataset = EEGWindowDataset(train_set)
test_dataset = EEGWindowDataset(test_set)

train_loader = DataLoader(train_dataset, batch_size=4096, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4096)

In [6]:
from torch import nn
from ncps.torch import CfC

n_channels = train_set[0][0].shape[0]  # e.g., 22
n_classes = 4  # e.g., 4

class EEGCfCNet(nn.Module):
    def __init__(self, input_size, hidden_size, n_classes):
        super().__init__()
        self.rnn = CfC(input_size, hidden_size)
        self.hidden_size = hidden_size
        self.classifier = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        # x: (batch, time, features)
        h0 = torch.zeros(x.size(0), self.hidden_size, device=x.device)
        out, _ = self.rnn(x, h0)
        # Use last time step output
        out = out[:, -1, :]
        return self.classifier(out)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EEGCfCNet(input_size=n_channels, hidden_size=64, n_classes=n_classes).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            loss = criterion(output, y)
            total_loss += loss.item() * y.size(0)
            preds = output.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    running_correct = 0
    running_total = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for X, y in loop:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * y.size(0)
        preds = output.argmax(dim=1)
        running_correct += (preds == y).sum().item()
        running_total += y.size(0)

        loop.set_postfix(train_loss=running_loss/running_total,
                         train_acc=running_correct/running_total)

    train_loss = running_loss / running_total
    train_acc = running_correct / running_total

    val_loss, val_acc = evaluate(model, test_loader, criterion)

    print(f"Epoch {epoch+1} summary: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/100:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 summary: Train Loss: 1.3952, Train Acc: 0.2326, Val Loss: 1.3954, Val Acc: 0.2384


Epoch 2/100:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2 summary: Train Loss: 1.3891, Train Acc: 0.2515, Val Loss: 1.3947, Val Acc: 0.2384


Epoch 3/100:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch 98 summary: Train Loss: 0.9895, Train Acc: 0.5860, Val Loss: 1.7961, Val Acc: 0.2519


Epoch 99/100:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 99 summary: Train Loss: 0.9820, Train Acc: 0.5976, Val Loss: 1.7927, Val Acc: 0.2488


Epoch 100/100:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 100 summary: Train Loss: 0.9628, Train Acc: 0.6053, Val Loss: 1.8035, Val Acc: 0.2531


In [8]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X, y in tqdm(test_loader):
        X, y = X.to(device), y.to(device)
        out = model(X)
        pred = out.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)
print(f"Test accuracy: {correct/total:.2%}")

  0%|          | 0/1 [00:00<?, ?it/s]

Test accuracy: 25.31%
